Doing this tutorial: https://nbconvert.readthedocs.io/en/latest/nbconvert_library.html

In [1]:
from urllib.request import urlopen

url = 'https://jakevdp.github.io/downloads/notebooks/XKCD_plots.ipynb'
response = urlopen(url).read().decode()
response[0:60] + ' ...'

'{\n "cells": [\n  {\n   "cell_type": "markdown",\n   "metadata": ...'

In [2]:
import nbformat
jake_notebook = nbformat.reads(response, as_version=4)
jake_notebook.cells[0]

{'cell_type': 'markdown',
 'metadata': {},
 'source': '# XKCD plots in Matplotlib'}

In [28]:
from traitlets.config import Config

# 1. Import the exporter
from nbconvert import HTMLExporter

# 2. Instantiate the exporter. We use the `classic` template for now; we'll get into more details
# later about how to customize the exporter further.
html_exporter = HTMLExporter(template_name='classic')

# 3. Process the notebook we loaded earlier
(body, resources) = html_exporter.from_notebook_node(jake_notebook)

In [18]:
print(body[:400] + '...')


<!DOCTYPE html>
<html>
<head><meta charset="utf-8" />
<meta name="viewport" content="width=device-width, initial-scale=1.0">

<title>Notebook</title><script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js"></script><script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>

<style type="text/css">
  pre { line-height: 125%; }
td.linenos ....


In [19]:
print("Resources:", resources.keys())
print("Metadata:", resources['metadata'].keys())
print("Inlining:", resources['inlining'].keys())
print("Extension:", resources['output_extension'])

Resources: dict_keys(['metadata', 'output_extension', 'deprecated', 'theme', 'include_css', 'include_js', 'include_url', 'require_js_url', 'mathjax_url', 'jquery_url', 'jupyter_widgets_base_url', 'widget_renderer_url', 'html_manager_semver_range', 'inlining', 'raw_mimetypes', 'global_content_filter'])
Metadata: dict_keys(['name'])
Inlining: dict_keys(['css'])
Extension: .html


In [140]:
from traitlets import Integer
from nbconvert.preprocessors import Preprocessor

class PelicanSubCell(Preprocessor):
    """A Pelican specific preprocessor to remove some of the cells of a notebook"""

    # I could also read the cells from nb.metadata.pelican if someone wrote a JS extension,
    # but for now I'll stay with configurable value.
    start = Integer(0,  help="first cell of notebook to be converted").tag(config=True)
    end   = Integer(-1, help="last cell of notebook to be converted").tag(config=True)

    def preprocess(self, nb, resources):
        self.log.info("I'll keep only cells from %d to %d", self.start, self.end)
        nb.cells = nb.cells[self.start:self.end]
        return nb, resources

In [143]:
# Create a new config object that configures both the new preprocessor, as well as the exporter
c =  Config()
c.PelicanSubCell.start = 3
c.PelicanSubCell.end = 6
c.RSTExporter.preprocessors = [PelicanSubCell]

# Create our new, customized exporter that uses our custom preprocessor
pelican = RSTExporter(config=c)

# Process the notebook
print(pelican.from_notebook_node(jake_notebook)[0])

.. code:: ipython3

    from IPython.display import Image
    Image('http://jakevdp.github.com/figures/xkcd_version.png')




.. image:: output_3_0.png



Sometimes when showing schematic plots, this is the type of figure I
want to display. But drawing it by hand is a pain: I’d rather just use
matplotlib. The problem is, matplotlib is a bit too precise. Attempting
to duplicate this figure in matplotlib leads to something like this:

.. code:: ipython3

    Image('http://jakevdp.github.com/figures/mpl_version.png')




.. image:: output_5_0.png





### High Level Notebook Conversion Process

From [this](https://nbconvert.readthedocs.io/en/latest/nbconvert_library.html#Some-theory)

Before we get into actually extracting the figures, it will be helpful to give a high-level overview of the process of converting a notebook to a another format:

1. Retrieve the notebook and it’s accompanying resources (you are responsible for this).

2. Feed the notebook into the `Exporter`, which:
    
    a. Sequentially feeds the notebook into an array of `Preprocessors`. Preprocessors only act on the structure of the notebook, and have unrestricted access to it.
    
    b. Feeds the notebook into the Jinja templating engine, which converts it to a particular format depending on which template is selected.
    
5. The exporter returns the converted notebook and other relevant resources as a tuple.

6. You write the data to the disk using the built-in `FilesWriter` (which writes the notebook and any extracted files to disk), or elsewhere using a custom `Writer`.

### Creating Template files on the fly: means you don't have to store templates.

In [25]:
from jinja2 import DictLoader

dl = DictLoader({'footer2':
"""
{%- extends 'lab/index.html.j2' -%}

{% block footer %}
FOOOOOOOOTEEEEER
{% endblock footer %}
"""})


exportHTML = HTMLExporter(extra_loaders=[dl], template_file='footer2')
(body, resources) = exportHTML.from_notebook_node(jake_notebook)
for l in body.split('\n')[-4:]:
    print(l)


</body>

FOOOOOOOOTEEEEER



## Aside: WTF is [Traitlets](https://traitlets.readthedocs.io/en/stable/using_traitlets.html)

In [54]:
from traitlets import HasTraits, Int, Unicode, default, TraitError
import getpass

class Identity(HasTraits):
    username = Unicode()

    @default('username')
    def _default_username(self):
        return getpass.getuser() # getpass is in the standard lib and getuser() gets the user name.

Two things going on here:

1. the default value of `username` will be set to whatever is returned by the thing decorated by `@default('username')`.  The method can be named anything, but the convention is `_default_<property_name>`.
2. There will be input validation such that a `Trait` error will be raised if the input passed is of a different type.

In [55]:
i = Identity() # the default value is used
assert i.username == getpass.getuser()

In [64]:
try:
    i = Identity(username=3)
except TraitError as e:
    print(e)

The 'username' trait of an Identity instance expected a unicode string, not the int 3.


### Observe:, like callbacks if properties change values

In [97]:
class Foo(HasTraits):
    bar = Int()
    baz = Unicode()

foo = Foo()

def func(change):
    print(change.old)
    print(change.new)   # as of traitlets 4.3, one should be able to
                           # write print(change.new) instead

In [98]:
foo.bar, foo.baz

(0, '')

In [99]:
foo.observe(func, names=['bar'])

In [100]:
foo.bar = 1  # prints '0\n 1'

0
1


In [101]:
foo.bar = 1 # doesn't print anything because nothing has changed.

In [102]:
foo.bar = 2  # prints '1\n 2'

1
2


In [103]:
foo.baz = 'abc'  # prints nothing b/c no observe thing

#### Fine, what about `traitlets.config.Config`

https://traitlets.readthedocs.io/en/stable/config.html

> This is the configuration system used by IPython and Jupyter.